In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import json
import math
from scipy import stats
import statsmodels.api as sm
import os
from fbprophet import Prophet
from statsmodels.distributions.empirical_distribution import ECDF
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from plotly import graph_objs as go
init_notebook_mode(connected=True)

In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\jayam\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
path=os.getcwd()
path

'C:\\Users\\jayam\\Downloads\\7879phd'

In [3]:
test=pd.read_csv(path +"\\test_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""],usecols=["date","id","city_medicine","year","discounted"])
train=pd.read_csv(path +"\\train_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""],usecols=["date","sales","city_medicine","year","discounted"])

#### Generally propht  model is used in time series data, let us see how it performs on our dataset 

In [4]:
train["date"]=pd.to_datetime(train["date"])
test["date"]=pd.to_datetime(test["date"])

#### Let us take a random medicines and check propht model performance

In [5]:
df=train[train["city_medicine"]=="7_1207"][["date","sales"]].reset_index(drop=True)

In [6]:
df.head()

,date,sales
0,2015-01-02,92.0
1,2015-01-03,176.0
2,2015-01-04,152.0
3,2015-01-05,184.0
4,2015-01-06,220.0


In [7]:
df.columns

Index(['date', 'sales'], dtype='object')

In [8]:
df.columns = ['ds', 'y']
df.tail(n=3)

,ds,y
1266,2018-06-28,36.0
1267,2018-06-29,64.0
1268,2018-06-30,40.0


In [9]:
df = df.loc[df.ds >= '2017-01-01']

In [10]:
df.tail(2)

,ds,y
1267,2018-06-29,64.0
1268,2018-06-30,40.0


In [11]:
prediction_size = 30
train_df = df[:-prediction_size]
train_df.tail()

,ds,y
1234,2018-05-27,152.0
1235,2018-05-28,120.0
1236,2018-05-29,96.0
1237,2018-05-30,52.0
1238,2018-05-31,100.0


In [12]:
train_df.head(2)

,ds,y
726,2017-01-02,220.0
727,2017-01-03,224.0


In [13]:
train_df.shape

(513, 2)

In [14]:
m = Prophet(interval_width = 0.95)
m.fit(train_df)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [15]:
future = m.make_future_dataframe(periods=30)
future[-30:]
forecast = m.predict(future[-30:])
forecast[["ds","yhat"]]

,ds,yhat
0,2018-06-01,45.618192
1,2018-06-02,53.728607
2,2018-06-03,49.674440
3,2018-06-04,74.628812
4,2018-06-05,113.312091
5,2018-06-06,89.258628
6,2018-06-07,50.880790
7,2018-06-08,45.170124
8,2018-06-09,53.280539
9,2018-06-10,49.226372


In [16]:
forecast = m.predict(future)
forecast.tail(n=3)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
540,2018-06-28,66.621131,-13.882454,111.558908,66.620630,66.621679,-17.084546,-17.084546,-17.084546,-17.084546,-17.084546,-17.084546,0.0,0.0,0.0,49.536585
541,2018-06-29,66.557121,-20.145195,107.182479,66.556596,66.557693,-22.731203,-22.731203,-22.731203,-22.731203,-22.731203,-22.731203,0.0,0.0,0.0,43.825918
542,2018-06-30,66.493111,-7.751165,112.554319,66.492563,66.493708,-14.556778,-14.556778,-14.556778,-14.556778,-14.556778,-14.556778,0.0,0.0,0.0,51.936333


In [17]:
def make_comparison_dataframe(historical, forecast):# making comparision function
    """Join the history with the forecast.
    
       The resulting dataset will contain columns 'yhat', 'yhat_lower', 'yhat_upper' and 'y'.
    """
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [18]:
cmp_df = make_comparison_dataframe(df, forecast)
cmp_df.tail(n=3)

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2018-06-28,49.536585,-13.882454,111.558908,36.0
2018-06-29,43.825918,-20.145195,107.182479,64.0
2018-06-30,51.936333,-7.751165,112.554319,40.0


In [19]:
 def calculate_forecast_errors(df, prediction_size):# fuction for mape rmse and mae
    """Calculate MAPE, RMSE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    
    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    rmse=np.sqrt(mean_squared_error(df["y"],df["yhat"]))
     # Recall that we held out the values of the last `prediction_size` days
    # in order to predict them and measure the quality of the model. 
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e'),"RMSE":rmse}

In [20]:
for err_name, err_value in calculate_forecast_errors(cmp_df, prediction_size).items():
    print(err_name, err_value)

MAPE 36.98726798777546
MAE 20.914363151120945
RMSE 31.692311396581218


In [21]:
df1=train[train["city_medicine"]=="7_1115"][["date","sales"]].reset_index(drop=True)
df1.columns = ['ds', 'y']
df1 = df1.loc[df1.ds >= '2017-01-01']
prediction_size=30
train_df1 = df1[:-prediction_size]
m1 = Prophet(interval_width = 0.95)
m1.fit(train_df1)
future1 = m1.make_future_dataframe(periods=prediction_size)
forecast1 = m1.predict(future1)
cmp_df1 = make_comparison_dataframe(df1, forecast1)
for err_name, err_value in calculate_forecast_errors(cmp_df1, prediction_size).items():
    print(err_name, err_value)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


MAPE 72.57192156170787
MAE 16.094958361734115
RMSE 20.912338375599617


In [22]:
forecast1

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2017-01-02,43.050330,-1.297972,78.052695,43.050330,43.050330,-5.926111,-5.926111,-5.926111,-5.926111,-5.926111,-5.926111,0.0,0.0,0.0,37.124218
1,2017-01-03,43.047048,11.456925,95.588764,43.047048,43.047048,10.403728,10.403728,10.403728,10.403728,10.403728,10.403728,0.0,0.0,0.0,53.450776
2,2017-01-04,43.043765,22.723757,103.597076,43.043765,43.043765,20.672249,20.672249,20.672249,20.672249,20.672249,20.672249,0.0,0.0,0.0,63.716014
3,2017-01-05,43.040483,1.160778,79.552702,43.040483,43.040483,-2.727962,-2.727962,-2.727962,-2.727962,-2.727962,-2.727962,0.0,0.0,0.0,40.312521
4,2017-01-06,43.037201,-1.313728,80.886073,43.037201,43.037201,-1.576714,-1.576714,-1.576714,-1.576714,-1.576714,-1.576714,0.0,0.0,0.0,41.460487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
538,2018-06-26,42.945224,11.623441,93.434317,42.916174,42.982260,10.403728,10.403728,10.403728,10.403728,10.403728,10.403728,0.0,0.0,0.0,53.348952
539,2018-06-27,42.954892,21.596729,103.248967,42.924561,42.993738,20.672249,20.672249,20.672249,20.672249,20.672249,20.672249,0.0,0.0,0.0,63.627141
540,2018-06-28,42.964560,3.800478,81.156911,42.932584,43.005289,-2.727962,-2.727962,-2.727962,-2.727962,-2.727962,-2.727962,0.0,0.0,0.0,40.236598
541,2018-06-29,42.974228,-1.949840,85.273958,42.940157,43.016976,-1.576714,-1.576714,-1.576714,-1.576714,-1.576714,-1.576714,0.0,0.0,0.0,41.397515


so we get forcast for entire train dataset

In [23]:
df1=train[(train["city_medicine"]=="7_1207")&(train["date"]>='2017-01-01')][["date","sales","discounted"]].reset_index(drop=True)
df1_temp=df1.copy()
df1.columns = ['ds', 'y',"discounted"]
df1 = df1.loc[df1.ds >= '2017-01-01']
prediction_size=30
train_df1 = df1[:-prediction_size]
m1 = Prophet(interval_width = 0.95)
m1.add_regressor('discounted')
#m1.add_regressor("week")
#m1.add_regressor("footfall_perday_permed")
m1.fit(train_df1)
future1 = m1.make_future_dataframe(periods=prediction_size)
future1["discounted"]=df1_temp["discounted"]
#future1["week"]=df1_temp["week"]
#future1["footfall_perday_permed"]=df1_temp["footfall_perday_permed"]
forecast1 = m1.predict(future1[-31:])
cmp_df1 = make_comparison_dataframe(df1, forecast1)
cmp_df1.fillna(0,inplace=True)
for err_name, err_value in calculate_forecast_errors(cmp_df1, prediction_size).items():
    print(err_name, err_value)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


MAPE 37.66742283168276
MAE 21.119726508218392
RMSE 28.63241373701282


In [276]:
cmp_df1

,yhat,yhat_lower,yhat_upper,y,discounted,week,footfall_perday_permed
ds,,,,,,,
2018-02-20,12.708824,4.424351,21.582726,8.0,0.0,8.0,0.098613
2018-02-21,11.692928,3.152473,20.246702,0.0,0.0,0.0,0.000000
2018-02-22,9.176815,0.655770,17.207096,0.0,0.0,0.0,0.000000
2018-02-23,22.909803,13.923159,31.377993,0.0,0.0,0.0,0.000000
2018-02-24,6.387865,-3.101526,15.267380,4.0,0.0,8.0,0.053138
2018-02-25,16.309522,7.998841,25.482895,4.0,0.0,8.0,0.063701
2018-02-26,11.013447,1.826947,18.709321,24.0,0.0,9.0,0.343541
2018-02-27,25.297957,16.721013,33.856083,0.0,0.0,0.0,0.000000
2018-02-28,19.144896,10.760364,27.724144,4.0,0.0,9.0,0.022911


In [259]:
forecast1

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,discounted,...,extra_regressors_additive,extra_regressors_additive_lower,extra_regressors_additive_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2018-02-20,19.511996,-9.434305,56.643460,19.511996,19.511996,1.912480,1.912480,1.912480,0.0,...,0.0,0.0,0.0,1.912480,1.912480,1.912480,0.0,0.0,0.0,21.424475
1,2018-02-21,19.521658,-8.251354,57.558441,19.521658,19.521658,4.431792,4.431792,4.431792,0.0,...,0.0,0.0,0.0,4.431792,4.431792,4.431792,0.0,0.0,0.0,23.953450
2,2018-02-22,19.531321,-14.518229,54.862126,19.531321,19.531321,-0.121592,-0.121592,-0.121592,0.0,...,0.0,0.0,0.0,-0.121592,-0.121592,-0.121592,0.0,0.0,0.0,19.409729
3,2018-02-23,19.540984,-20.033261,50.563260,19.540984,19.540984,-2.839011,-2.839011,-2.839011,0.0,...,0.0,0.0,0.0,-2.839011,-2.839011,-2.839011,0.0,0.0,0.0,16.701973
4,2018-02-24,19.550646,-9.930872,58.157586,19.550646,19.550646,3.944076,3.944076,3.944076,0.0,...,0.0,0.0,0.0,3.944076,3.944076,3.944076,0.0,0.0,0.0,23.494722
5,2018-02-25,19.560309,-16.912704,47.861600,19.560309,19.560309,-5.142636,-5.142636,-5.142636,0.0,...,0.0,0.0,0.0,-5.142636,-5.142636,-5.142636,0.0,0.0,0.0,14.417673
6,2018-02-26,19.569972,-15.609960,50.408292,19.569971,19.569972,-2.185108,-2.185108,-2.185108,0.0,...,0.0,0.0,0.0,-2.185108,-2.185108,-2.185108,0.0,0.0,0.0,17.384863
7,2018-02-27,19.579634,-14.014830,56.316225,19.579634,19.579634,1.912480,1.912480,1.912480,0.0,...,0.0,0.0,0.0,1.912480,1.912480,1.912480,0.0,0.0,0.0,21.492114
8,2018-02-28,19.589297,-8.248670,54.310534,19.589297,19.589297,4.431792,4.431792,4.431792,0.0,...,0.0,0.0,0.0,4.431792,4.431792,4.431792,0.0,0.0,0.0,24.021089
9,2018-03-01,19.598959,-14.259073,51.668904,19.598959,19.598960,-0.121592,-0.121592,-0.121592,0.0,...,0.0,0.0,0.0,-0.121592,-0.121592,-0.121592,0.0,0.0,0.0,19.477368


In [31]:
def show_forecast(cmp_df, num_predictions, num_values, title):
    """Visualize the forecast."""
    
    def create_go(name, column, num, **kwargs):
        points = cmp_df.tail(num)
        args = dict(name=name, x=points.index, y=points[column], mode='lines')
        args.update(kwargs)
        return go.Scatter(**args)
    
    lower_bound = create_go('Lower Bound', 'yhat_lower', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="gray"))
    upper_bound = create_go('Upper Bound', 'yhat_upper', num_predictions,
                            line=dict(width=0),
                            marker=dict(color="gray"),
                            fillcolor='rgba(68, 68, 68, 0.3)', 
                            fill='tonexty')
    forecast = create_go('Forecast', 'yhat', num_predictions,
                         line=dict(color='rgb(31, 119, 180)'))
    actual = create_go('Actual', 'y', num_values,marker=dict(color="red"))
    
    # In this case the order of the series is important because of the filling
    data = [lower_bound, upper_bound, forecast, actual]

    layout = go.Layout(yaxis=dict(title='Sales'), title=title, showlegend = False)
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

show_forecast(cmp_df, prediction_size, 100, '7_1207 medicine forecast')

#### Forecast looks better in propht model let us implement this for our dataset